In [1]:
using Pkg
Pkg.activate(@__DIR__)
push!(LOAD_PATH, joinpath(@__DIR__, "..", ".."))
Pkg.instantiate()

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `ssh://git@github.com/JuliaRegistries/General.git`


In [2]:
using JuMP
using LinearAlgebra
using MINLPTrajOpt
using Rotations
using RigidBodyDynamics

In [3]:
urdf = joinpath(dirname(pathof(RigidBodyDynamics)), "..", "test", "urdf", "Acrobot.urdf")
mechanism = parse_urdf(urdf)

Spanning tree:
Vertex: world (root)
  Vertex: upper_link, Edge: shoulder
    Vertex: lower_link, Edge: elbow
No non-tree joints.

In [54]:
push!(LOAD_PATH, joinpath(@__DIR__, "..", "..", "extras", "TrigonometricPolynomials"))
using TrigonometricPolynomials
using TrigonometricPolynomials: Var

In [55]:
sincosmap = SinCosDict()
q = [TrigPoly{Float64}(Var("q$i"), sincosmap) for i = 1 : num_positions(mechanism)]
v = [TrigPoly{Float64}(Var("v$i"), sincosmap) for i = 1 : num_velocities(mechanism)]
state = MechanismState(mechanism, q, v)
setdirty!(state)

In [83]:
M = mass_matrix(state)
map!(simplify, M.data, M.data)
M

2×2 Symmetric{TrigPoly{Float64},Array{TrigPoly{Float64},2}}:
           2.0cos(q2) + 2.663  cos(q2) + 1.33
 cos(q2) + 1.33                          1.33

In [84]:
c = dynamics_bias(state)
map!(simplify ∘ simplify, c, c)
c

2-element SegmentedVector{JointID,TrigPoly{Float64},Base.OneTo{JointID},Array{TrigPoly{Float64},1}}:
             -2.0v1v2sin(q2) - v2²sin(q2) + 9.81sin(q1)cos(q2) + 9.81cos(q1)sin(q2) + 14.715sin(q1)
 v1²sin(q2) + 9.81sin(q1)cos(q2) + 9.81cos(q1)sin(q2)                                              

In [81]:
c[1]

-2.0v1v2sin(q2) - v2²sin(q2) + 9.81sin(q1)cos(q2) + 9.81cos(q1)sin(q2) + 14.715sin(q1)

Design decisions:

* `Problem` type:
  * stores a `JuMP.Model`
  * does all the setup in the constructor
  * stores variables/useful subexpressions
  * solve method
  * methods for accessing results
  * use `AxisArrays` to store variables
* No separate `Result` type
* Implement `TrigPoly` type, run through `mass_matrix` and `dynamics_bias`.

Is this really needed?